<a href="https://colab.research.google.com/github/adari-git1/python/blob/master/iphone-data-analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [103]:
pip install pyspark

In [104]:
from pyspark.sql import SparkSession

In [105]:
spark=SparkSession.builder.appName('Fitness_tracker').getOrCreate()

In [106]:
data=spark.read.option('header',True).csv('/content/Fitness_trackers.csv')

In [107]:
data.show()

+----------+-----------+--------------------+--------------------+-------------+--------------+--------------+-----------------+--------------------+------------------------------+-------+
|Brand Name|Device Type|          Model Name|               Color|Selling Price|Original Price|       Display|Rating (Out of 5)|      Strap Material|Average Battery Life (in days)|Reviews|
+----------+-----------+--------------------+--------------------+-------------+--------------+--------------+-----------------+--------------------+------------------------------+-------+
|    Xiaomi|FitnessBand|        Smart Band 5|               Black|        2,499|         2,999|AMOLED Display|              4.1|Thermoplastic pol...|                            14|   NULL|
|    Xiaomi|FitnessBand|        Smart Band 4|               Black|        2,099|         2,499|AMOLED Display|              4.2|Thermoplastic pol...|                            14|   NULL|
|    Xiaomi|FitnessBand|            HMSH01GE|          

In [108]:
data.createOrReplaceTempView('Table1')

In [109]:
spark.sql("select count(distinct `Brand Name`) from Table1").show()

+--------------------------+
|count(DISTINCT Brand Name)|
+--------------------------+
|                        20|
+--------------------------+



In [110]:
data.write.mode('overwrite').partitionBy('Brand Name').format('csv').save('data2')

In [143]:
Apple=spark.read.csv('/content/data2/Brand Name=APPLE/part-00000-2b5f8810-e3bd-4194-9da5-ee68fa90c0cb.c000.csv')

In [144]:
Apple=Apple.withColumnRenamed("_c0", "Device Type").withColumnRenamed("_c1", "Model name").withColumnRenamed("_c2", "Color").withColumnRenamed("_c3", "Selling_Price").withColumnRenamed("_c4", "Original_price").withColumnRenamed("_c5", "Display").withColumnRenamed("_c6", "Rating").withColumnRenamed("_c7", "Strap_material").withColumnRenamed("_c8", "Battery_life")

In [145]:
Apple=Apple.drop('_c9')

In [146]:
Apple.show()

+-----------+--------------------+------------------+-------------+--------------+-------------------+------+---------------+------------+
|Device Type|          Model name|             Color|Selling_Price|Original_price|            Display|Rating| Strap_material|Battery_life|
+-----------+--------------------+------------------+-------------+--------------+-------------------+------+---------------+------------+
| Smartwatch|Series 3 GPS - 42...|             Black|       18,900|        23,900|OLED Retina Display|   4.6|      Aluminium|           1|
| Smartwatch|SE 44 mm Space Gr...|             Black|       30,400|        32,900|OLED Retina Display|   4.7|      Aluminium|           1|
| Smartwatch|Series 3 GPS - 38...|             Black|       16,900|        20,900|OLED Retina Display|   4.6|      Aluminium|           1|
| Smartwatch|SE GPS + Cellular...|             Black|       36,900|        36,900|OLED Retina Display|   4.6|      Aluminium|           1|
| Smartwatch|SE GPS + Cellu

In [116]:
Apple=Apple.replace({',':''})

In [160]:
Apple=Apple.withColumn('Selling_Price',col('Selling_price').cast('int')).withColumn('Original_price',col('Original_price').cast('int'))

In [161]:
Apple.printSchema()

root
 |-- Device Type: string (nullable = true)
 |-- Model name: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Selling_Price: integer (nullable = true)
 |-- Original_price: integer (nullable = true)
 |-- Display: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Strap_material: string (nullable = true)
 |-- Battery_life: string (nullable = true)



In [162]:
Apple.createOrReplaceTempView('Table2')

In [163]:
spark.sql('select * from Table2').show()

+-----------+--------------------+------------------+-------------+--------------+-------------------+------+---------------+------------+
|Device Type|          Model name|             Color|Selling_Price|Original_price|            Display|Rating| Strap_material|Battery_life|
+-----------+--------------------+------------------+-------------+--------------+-------------------+------+---------------+------------+
| Smartwatch|Series 3 GPS - 42...|             Black|        18900|         23900|OLED Retina Display|   4.6|      Aluminium|           1|
| Smartwatch|SE 44 mm Space Gr...|             Black|        30400|         32900|OLED Retina Display|   4.7|      Aluminium|           1|
| Smartwatch|Series 3 GPS - 38...|             Black|        16900|         20900|OLED Retina Display|   4.6|      Aluminium|           1|
| Smartwatch|SE GPS + Cellular...|             Black|        36900|         36900|OLED Retina Display|   4.6|      Aluminium|           1|
| Smartwatch|SE GPS + Cellu

In [172]:
spark.sql('select `Device Type`,Display,rating,Discount from(select *,Original_price-Selling_Price as discount from Table2)Dis order by discount desc limit 1').show(truncate =False)

+-----------+-------------------+------+--------+
|Device Type|Display            |rating|Discount|
+-----------+-------------------+------+--------+
|Smartwatch |OLED Retina Display|4.5   |9910    |
+-----------+-------------------+------+--------+

